# 1Dイジング模型

ここでは次の1次元強磁性ハイゼンベルグ模型を考えます。

$$
\mathcal{H} = -\sum_i S_i S_{i+1}.
$$

前章の解説に従って、$i$番目のスピンを更新するとき、
2つの状態間のエネルギー差は$\Delta E = E_\uparrow - E_\downarrow = - 2h$
と計算出来ます。
ここで、$h = S_{i-1} + S_{i+1}$は$i$番目のサイト以外のスピンが、$i$番目のスピンに及ぼす有効磁場です。
最後に、$1/(1+\exp(\beta \Delta E ))$の確率で、$S_i=1$を採択します。

なお、指数関数の計算は重いので（かけ算とかに比べ）、予め指数関数の値は計算して保存しておきます。

In [2]:
@show VERSION
using BenchmarkTools, Random

VERSION = v"1.5.2"


## 最初の実装
以下の実装を見てみましょう。ising1d!という関数は、スピンあたりniters回のスピン更新を行います。
関数は最初に呼び出したにコンパイルされます。
実行速度を正しく計測するには、一回呼び出してからにしましょう。
BenchmarkTools中の@benchmarkを使えば、自動的に複数回呼び出して計測してくれます。

### 演習問題

毎回指数関数の値を計算する場合、どの程度性能が悪化するか確認してみましょう。

In [8]:
function ising1d!(s, β, niters, rng)
    n = length(s)
    min_h = -2
    max_h = 2
    prob = [1/(1+exp(-2*β*h)) for h in min_h:max_h]
    for iter in 1:niters, i in 1:n
        sl = s[ifelse(i == 1, n, i-1)]
        sr = s[ifelse(i == n, 1, i+1)]
        # h = -2, 0, 2
        h = sl + sr
        si_old = s[i]
        s[i] = ifelse(rand(rng) < prob[h-min_h+1], +1, -1)
    end
end

n = 100
rng = MersenneTwister(4649)
s0 = rand(rng, Int8[-1, 1], n)
β = 100.0
niters = 10^3

s = copy(s0)

# Run once to compile the function
ising1d!(s, β, niters, rng)

@time ising1d!(s, β, niters, rng)
@benchmark ising1d!(s, β, niters, rng) setup=(s = copy(s0))

  0.000545 seconds (1 allocation: 128 bytes)


BenchmarkTools.Trial: 
  memory estimate:  128 bytes
  allocs estimate:  1
  --------------
  minimum time:     414.057 μs (0.00% GC)
  median time:      433.385 μs (0.00% GC)
  mean time:        465.491 μs (0.00% GC)
  maximum time:     1.232 ms (0.00% GC)
  --------------
  samples:          10000
  evals/sample:     1

## 物理量計測込みの実装

では、次に物理量（磁化、エネルギー)の計算を含めたプログラムを書いてみます。
ここで重要な点は、磁化、エネルギーの値は、スピン状態の更新と同時に、差分で更新する点です。
この更新は、(スピン更新と同じ)$O(1)$の計算量ですみますが、
定義に従って最初から計算し直すと$O(N)$の計算量がかかってしまいます。

以下のコードはmodule MCの中に囲っています。
moduleの中に入れない場合、構造体structの再定義が許されないため、
コードを書くときの試行錯誤の妨げになるからです。

SpinStateはスピン状態を表すstructで、磁化、エネルギーの値も同時に保存しています。
スピン状態やこれらの物理量が常にconsistentであることを保証して計算を進めます。

エネルギー、磁化を計算する関数をそれぞれ定義し、SpinStateオブジェクトの生成、
状態更新前後のconsistencyのチェックに利用しています。

In [4]:
# Wrap everything with a module to allow redefition of type
module MC

"""
Composite type to represent a spin state
"""
mutable struct SpinState
    num_spins::Int
    s::Array{Int8,1}
    energy::Int
    tot_mag::Int
end

"""
Energy
"""
function energy(s)
    n = length(s)
    - sum((s[i] * s[ifelse(i == n, 1, i+1)] for i in 1:n))
end

"""
Total magnetization
"""
total_magnetization(s) = sum(s)

"""
Constructor
"""
function SpinState(s)
    ss = SpinState(length(s), copy(s), energy(s), total_magnetization(s))
    sanity_check(ss)
    ss
end

"""
Sanity check
"""
function sanity_check(ss)
    @assert energy(ss.s) == ss.energy
    @assert total_magnetization(ss.s) == ss.tot_mag
end

"""
Take an object of SpinState as an input and update it in place.
"""
function update!(ss, β, niters, rng)
    min_h = -2
    max_h = 2
    s = ss.s
    n = ss.num_spins
    prob = [1/(1+exp(-2*β*h)) for h in min_h:max_h]
    for iter in 1:niters, i in 1:n
        sl = s[ifelse(i == 1, n, i-1)]
        sr = s[ifelse(i == n, 1, i+1)]
        # h = -2, 0, 2
        h = sl + sr
        si_old = s[i]
        s[i] = ifelse(rand(rng) < prob[h-min_h+1], +1, -1)
        
        # Update observables with O(1) operations
        ss.energy += (si_old - s[i]) * h
        ss.tot_mag += (s[i] - si_old)
    end
end

end
;

OK, create an updater object and run it, and check if a spin state is updated correctly.

In [5]:
ss = MC.SpinState(s0)
MC.update!(ss, β, niters, rng)
MC.sanity_check(ss)

In [6]:
@benchmark MC.update!(ss, β, niters, rng)

BenchmarkTools.Trial: 
  memory estimate:  128 bytes
  allocs estimate:  1
  --------------
  minimum time:     436.042 μs (0.00% GC)
  median time:      467.521 μs (0.00% GC)
  mean time:        474.420 μs (0.00% GC)
  maximum time:     1.285 ms (0.00% GC)
  --------------
  samples:          10000
  evals/sample:     1

## Exercise
Implement measurement of the magnetization and specific heat 
and compare the results with the exact ones!
(To do) Include analytic expressions